In [21]:
import os
import sys
import csv
import requests
import json
import operator
import time
import datetime
import platform
import itertools

In [26]:
# !pip install gensim
# !pip3 install https://github.com/JonathanRaiman/glove/archive/master.zip

In [2]:
def writeText(text, path, mode = 'a'):
    with open (path, mode) as textout:
        textout.write((text))
        
def writeJson(json, path, mode = 'w'):
    with open(path, mode) as file:
        file.write(json.dumps(json))
        
def writeCsv(listOut, outputFile):
    import csv
    with open (outputFile, "w", newline='') as outputfile:
        writer = csv.writer(outputfile, delimiter = ",")
        for element in listOut:
            writer.writerow(element)

In [3]:
def getTxt(path):
    return open(path, 'r').read()

def getCsv(path, delim = ','):
    list_return = []
    with open (path) as file:
        csvreader = csv.reader(file, delimiter = delim)        
        for i, line in enumerate(csvreader):
            list_inner = []
            list_inner.append(line[0].replace('\"','').replace('\'','').replace('[','').replace(']','').strip().split(','))
            list_inner.append(line[1].replace('\"','').replace('\'','').replace('[','').replace(']','').strip().split(','))
            list_inner.append(line[2].replace('\"','').replace('\'','').replace('[','').replace(']','').strip().split(','))
            list_return.append(list_inner)
    
    
    return list_return

In [4]:
if platform.system() == 'Windows':
    prep_data = getCsv('..\\output\\prep_out.csv')
else:
    prep_data = getCsv('../output/prep_out.csv')

In [28]:
prep_data[:3]

[[['domestic',
   ' general',
   ' government',
   ' health',
   ' expenditure',
   ' capita',
   ' current',
   ' us',
   ' $',
   ' countries'],
  ['public',
   ' expenditure',
   ' health',
   ' domestic',
   ' sources',
   ' capita',
   ' expressed',
   ' current',
   ' us',
   ' dollars'],
  ['social issues',
   ' politics',
   ' business_finance',
   ' environment',
   ' economy',
   ' national accounts',
   ' health systems by country',
   ' military budget',
   ' world bank domestic']],
 [['domestic',
   ' general',
   ' government',
   ' health',
   ' expenditure',
   ' capita',
   ' current',
   ' us',
   ' $',
   ' world',
   ' regions'],
  ['public',
   ' expenditure',
   ' health',
   ' domestic',
   ' sources',
   ' capita',
   ' expressed',
   ' current',
   ' us',
   ' dollars'],
  ['social issues',
   ' environment',
   ' business_finance',
   ' politics',
   ' economy',
   ' national accounts',
   ' health systems by country',
   ' military budget',
   ' world bank do

## Word2Vect

In [13]:
# from gensim.models import word2vec
# import logging

In [29]:
import glove
from gensim.models.word2vec import Text8Corpus
from glove import Corpus, Glove

ModuleNotFoundError: No module named 'glove'

In [27]:
sentences = list(itertools.islice(Text8Corpus('text8'),None))
corpus = Corpus()
corpus.fit(sentences, window=10)
glove = Glove(no_components=100, learning_rate=0.05)

glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

FileNotFoundError: [Errno 2] No such file or directory: 'text8'